In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
import string
import re
%matplotlib inline

In [ ]:
xlsx = pd.ExcelFile('depressed.xlsx')

In [ ]:
data = pd.read_excel(xlsx, 'RapidMiner Data',header=0)
data

,Created-At,From-User,From-User-Id,To-User,To-User-Id,Language,Source,Text,Geo-Location-Latitude,Geo-Location-Longitude,Retweet-Count,Id
0,2021-10-03 23:00:33,Michelle Obama,409486555,NaN,-1,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",How it started vs. how it’s going\n\nHappy ann...,NaN,NaN,23025,1444693864275709958
1,2021-10-04 07:24:12,BrooklynDad_Defiant!,1640929196,NaN,-1,en,"<a href=""http://twitter.com/download/android"" ...",Who else would LOVE to see the media relentles...,NaN,NaN,8552,1444820612988522496
2,2021-10-05 08:43:45,SMTOWN,884378714,NaN,-1,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",TAEIL of NCT & rapper Lil Boi to feature in Ra...,NaN,NaN,15586,1445203019553861633
3,2021-10-05 10:39:36,???? ????????xy ⁷,1045314656863023112,NaN,-1,en,"<a href=""http://twitter.com/download/iphone"" r...",RT @chartdata: All of @BTS_twt's Hot 100 #1’s:...,NaN,NaN,10785,1445232173934469125
4,2021-10-05 10:39:35,Amanda Evans,1248046770682290178,NaN,-1,en,"<a href=""http://twitter.com/download/android"" ...",RT @AdamsCumbie: I love this! ?????? https://t...,NaN,NaN,100,1445232173095731204
...,...,...,...,...,...,...,...,...,...,...,...,...
1986,2021-10-03 23:00:00,Decadent Publishing,142780741,NaN,-1,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Will Tiffany and Charles be able to save their...,NaN,NaN,1,1444693726681575425
1987,2021-10-03 23:00:00,Terese McGroarty,22322884,NaN,-1,en,"<a href=""http://twitter.com/download/iphone"" r...",RT @JoelOsteen: I love the fact that Jesus bro...,NaN,NaN,1873,1444693726677372941
1988,2021-10-03 23:00:00,ACT:⁵Acehilnr_?? |??STREAM??,1379126745815654403,NaN,-1,en,"<a href=""http://twitter.com/download/android"" ...",RT @joe_txtmoa: Bighit is no longer holding ba...,NaN,NaN,558,1444693726673072134
1989,2021-10-03 23:00:00,?? ?? ??,1017947223164882944,NaN,-1,en,"<a href=""http://twitter.com/download/iphone"" r...",love love love ?? https://t.co/bdULnKntMC,NaN,NaN,0,1444693726643822597


## Exploratory Data Anaysis: EDA

In [ ]:
# Drop unrelated columns
data = data.drop(['Created-At','Id','From-User', 'From-User-Id', 'Language', 'To-User', 'To-User-Id', 'Source', 'Geo-Location-Latitude', 'Geo-Location-Longitude','Retweet-Count'], axis=1)

In [ ]:
data.head()

,Text
0,How it started vs. how it’s going\n\nHappy ann...
1,Who else would LOVE to see the media relentles...
2,TAEIL of NCT & rapper Lil Boi to feature in Ra...
3,RT @chartdata: All of @BTS_twt's Hot 100 #1’s:...
4,RT @AdamsCumbie: I love this! ?????? https://t...


In [ ]:
print('Data size: ', data.shape)
print('Columns: ', data.columns)

Data size:  (1991, 1)
Columns:  Index(['Text'], dtype='object')


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1991 entries, 1444693864275709958 to 1444693726614466568
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Created-At  1991 non-null   datetime64[ns]
 1   Text        1991 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 46.7+ KB


## Data Preprocessing

In [ ]:
# Find and drop duplicated data
data.Text.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
1986    False
1987    False
1988     True
1989    False
1990    False
Name: Text, Length: 1991, dtype: bool

In [ ]:
data.Text.duplicated().sum()

425

In [ ]:
(~data.Text.duplicated()).sum()

1566

In [ ]:
data.drop_duplicates(inplace=True, subset='Text', keep='last')

In [ ]:
data

,Text
0,How it started vs. how it’s going\n\nHappy ann...
1,Who else would LOVE to see the media relentles...
2,TAEIL of NCT & rapper Lil Boi to feature in Ra...
3,RT @chartdata: All of @BTS_twt's Hot 100 #1’s:...
4,RT @AdamsCumbie: I love this! ?????? https://t...
...,...
1986,Will Tiffany and Charles be able to save their...
1987,RT @JoelOsteen: I love the fact that Jesus bro...
1988,RT @joe_txtmoa: Bighit is no longer holding ba...
1989,love love love ?? https://t.co/bdULnKntMC


In [ ]:
data.reset_index(drop=True,inplace=True)

In [ ]:
list1 = []

for i in data.loc[:,'Text']:
  t1 = " ".join(filter(lambda x:x[0]!= '@', i.split()))
  t2 = " ".join(filter(lambda x:x[0]!= '#', t1.split()))
  t3 = re.sub(r"http\S+", "", t2)
  text = t3.replace('RT','')
  list1.append(text)
  

In [ ]:
print(type(text))

<class 'str'>


In [ ]:
df = pd.DataFrame(list1)

In [ ]:
df

,0
0,How it started vs. how it’s going Happy annive...
1,Who else would LOVE to see the media relentles...
2,TAEIL of NCT & rapper Lil Boi to feature in Ra...
3,All of Hot 100 Dynamite Savage Love (Laxed - ...
4,I love this! ??????
...,...
1561,Will Tiffany and Charles be able to save their...
1562,I love the fact that Jesus broke the rules to...
1563,Bighit is no longer holding back!!! Theyre fl...
1564,love love love ??


In [ ]:
data

,Text
0,How it started vs. how it’s going\n\nHappy ann...
1,Who else would LOVE to see the media relentles...
2,TAEIL of NCT & rapper Lil Boi to feature in Ra...
3,RT @chartdata: All of @BTS_twt's Hot 100 #1’s:...
4,RT @AdamsCumbie: I love this! ?????? https://t...
...,...
1561,Will Tiffany and Charles be able to save their...
1562,RT @JoelOsteen: I love the fact that Jesus bro...
1563,RT @joe_txtmoa: Bighit is no longer holding ba...
1564,love love love ?? https://t.co/bdULnKntMC


In [ ]:
dataframe = pd.concat([data, df], ignore_index= True, axis= 1)


In [ ]:
dataframe[1]

0       How it started vs. how it’s going Happy annive...
1       Who else would LOVE to see the media relentles...
2       TAEIL of NCT & rapper Lil Boi to feature in Ra...
3        All of Hot 100 Dynamite Savage Love (Laxed - ...
4                                    I love this! ?????? 
                              ...                        
1561    Will Tiffany and Charles be able to save their...
1562     I love the fact that Jesus broke the rules to...
1563     Bighit is no longer holding back!!! Theyre fl...
1564                                   love love love ?? 
1565    Happy birthday to the most beautiful man I hav...
Name: 1, Length: 1566, dtype: object

In [ ]:
dataframe.rename(columns={0: 'text', 1: 'text_clean'}, inplace= True)

In [ ]:
dataframe['text_clean']

0       How it started vs. how it’s going Happy annive...
1       Who else would LOVE to see the media relentles...
2       TAEIL of NCT & rapper Lil Boi to feature in Ra...
3        All of Hot 100 Dynamite Savage Love (Laxed - ...
4                                    I love this! ?????? 
                              ...                        
1561    Will Tiffany and Charles be able to save their...
1562     I love the fact that Jesus broke the rules to...
1563     Bighit is no longer holding back!!! Theyre fl...
1564                                   love love love ?? 
1565    Happy birthday to the most beautiful man I hav...
Name: text_clean, Length: 1566, dtype: object

In [ ]:
dataframe['text_clean'].str.extractall(r'(\#\w+)')[0].value_counts()

#HappyOctoberBirthday    1
#lovemeharder            1
#JOSHUA                  1
#Chihuly                 1
#Likee                   1
#ForceFactor             1
#EmmaRose                1
#GirlsPlanet999          1
#glutenfree              1
Name: 0, dtype: int64

#### Remove punctuations

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def drop_punctuation(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    
    return text

In [ ]:
dataframe['Text_punct'] = dataframe['text_clean'].apply(lambda i: drop_punctuation(i))
dataframe.head()

,text,text_clean,Text_punct
0,How it started vs. how it’s going\n\nHappy ann...,How it started vs. how it’s going Happy annive...,How it started vs how it’s going Happy anniver...
1,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...
2,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT rapper Lil Boi to feature in Rai...
3,RT @chartdata: All of @BTS_twt's Hot 100 #1’s:...,All of Hot 100 Dynamite Savage Love (Laxed - ...,All of Hot Dynamite Savage Love Laxed Siren...
4,RT @AdamsCumbie: I love this! ?????? https://t...,I love this! ??????,I love this


#### Tokenization - Converting a sentence into list of words

In [ ]:
def token(text):
    text = re.split('\W+', text)
    return text

In [ ]:
dataframe['Text_token'] = dataframe['Text_punct'].apply(lambda i: token(i.lower()))
dataframe.head()

,text,text_clean,Text_punct,Text_token
0,How it started vs. how it’s going\n\nHappy ann...,How it started vs. how it’s going Happy annive...,How it started vs how it’s going Happy anniver...,"[how, it, started, vs, how, it, s, going, happ..."
1,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...,"[who, else, would, love, to, see, the, media, ..."
2,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT rapper Lil Boi to feature in Rai...,"[taeil, of, nct, rapper, lil, boi, to, feature..."
3,RT @chartdata: All of @BTS_twt's Hot 100 #1’s:...,All of Hot 100 Dynamite Savage Love (Laxed - ...,All of Hot Dynamite Savage Love Laxed Siren...,"[, all, of, hot, dynamite, savage, love, laxed..."
4,RT @AdamsCumbie: I love this! ?????? https://t...,I love this! ??????,I love this,"[, i, love, this, ]"


#### Remove stopwords

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stopword = nltk.corpus.stopwords.words('english')
stopword

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [ ]:
dataframe['Text_stopword'] = dataframe['Text_token'].apply(lambda i: remove_stopwords(i))
dataframe.head()

,text,text_clean,Text_punct,Text_token,Text_stopword
0,How it started vs. how it’s going\n\nHappy ann...,How it started vs. how it’s going Happy annive...,How it started vs how it’s going Happy anniver...,"[how, it, started, vs, how, it, s, going, happ...","[started, vs, going, happy, anniversary, barac..."
1,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...,"[who, else, would, love, to, see, the, media, ...","[else, would, love, see, media, relentlessly, ..."
2,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT rapper Lil Boi to feature in Rai...,"[taeil, of, nct, rapper, lil, boi, to, feature...","[taeil, nct, rapper, lil, boi, feature, raiden..."
3,RT @chartdata: All of @BTS_twt's Hot 100 #1’s:...,All of Hot 100 Dynamite Savage Love (Laxed - ...,All of Hot Dynamite Savage Love Laxed Siren...,"[, all, of, hot, dynamite, savage, love, laxed...","[, hot, dynamite, savage, love, laxed, siren, ..."
4,RT @AdamsCumbie: I love this! ?????? https://t...,I love this! ??????,I love this,"[, i, love, this, ]","[, love, ]"


#### Lammatization/ Stemming - Tranforming any form of a word to its root word

In [ ]:
poster = nltk.PorterStemmer()

In [ ]:
def stemming(text):
    text = [poster.stem(word) for word in text]
    return text

In [ ]:
dataframe['Text_stemming'] = dataframe['Text_stopword'].apply(lambda i: stemming(i))
dataframe.head()

,text,text_clean,Text_punct,Text_token,Text_stopword,Text_stemming
0,How it started vs. how it’s going\n\nHappy ann...,How it started vs. how it’s going Happy annive...,How it started vs how it’s going Happy anniver...,"[how, it, started, vs, how, it, s, going, happ...","[started, vs, going, happy, anniversary, barac...","[start, vs, go, happi, anniversari, barack, lo..."
1,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...,"[who, else, would, love, to, see, the, media, ...","[else, would, love, see, media, relentlessly, ...","[els, would, love, see, media, relentlessli, q..."
2,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT rapper Lil Boi to feature in Rai...,"[taeil, of, nct, rapper, lil, boi, to, feature...","[taeil, nct, rapper, lil, boi, feature, raiden...","[taeil, nct, rapper, lil, boi, featur, raiden,..."
3,RT @chartdata: All of @BTS_twt's Hot 100 #1’s:...,All of Hot 100 Dynamite Savage Love (Laxed - ...,All of Hot Dynamite Savage Love Laxed Siren...,"[, all, of, hot, dynamite, savage, love, laxed...","[, hot, dynamite, savage, love, laxed, siren, ...","[, hot, dynamit, savag, love, lax, siren, beat..."
4,RT @AdamsCumbie: I love this! ?????? https://t...,I love this! ??????,I love this,"[, i, love, this, ]","[, love, ]","[, love, ]"


In [ ]:
wordnet = nltk.WordNetLemmatizer()

In [ ]:
def lemmatizer(text):
    text = [wordnet.lemmatize(word) for word in text]
    return text

In [ ]:
dataframe['Text_lammetization'] = dataframe['Text_stopword'].apply(lambda i: lemmatizer(i))
dataframe.head()

,text,text_clean,Text_punct,Text_token,Text_stopword,Text_stemming,Text_lammetization
0,How it started vs. how it’s going\n\nHappy ann...,How it started vs. how it’s going Happy annive...,How it started vs how it’s going Happy anniver...,"[how, it, started, vs, how, it, s, going, happ...","[started, vs, going, happy, anniversary, barac...","[start, vs, go, happi, anniversari, barack, lo...","[started, v, going, happy, anniversary, barack..."
1,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...,Who else would LOVE to see the media relentles...,"[who, else, would, love, to, see, the, media, ...","[else, would, love, see, media, relentlessly, ...","[els, would, love, see, media, relentlessli, q...","[else, would, love, see, medium, relentlessly,..."
2,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT & rapper Lil Boi to feature in Ra...,TAEIL of NCT rapper Lil Boi to feature in Rai...,"[taeil, of, nct, rapper, lil, boi, to, feature...","[taeil, nct, rapper, lil, boi, feature, raiden...","[taeil, nct, rapper, lil, boi, featur, raiden,...","[taeil, nct, rapper, lil, boi, feature, raiden..."
3,RT @chartdata: All of @BTS_twt's Hot 100 #1’s:...,All of Hot 100 Dynamite Savage Love (Laxed - ...,All of Hot Dynamite Savage Love Laxed Siren...,"[, all, of, hot, dynamite, savage, love, laxed...","[, hot, dynamite, savage, love, laxed, siren, ...","[, hot, dynamit, savag, love, lax, siren, beat...","[, hot, dynamite, savage, love, laxed, siren, ..."
4,RT @AdamsCumbie: I love this! ?????? https://t...,I love this! ??????,I love this,"[, i, love, this, ]","[, love, ]","[, love, ]","[, love, ]"


In [ ]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text = re.sub('[0-9]+', '', text)
    text = re.split('\W+', text)    # tokenization
    text = [poster.stem(word) for word in text if word not in stopword]  # remove stopwords and stemming
    text = [wordnet.lemmatize(word) for word in text] #lemmatizer
    return text

In [ ]:
countVectorizer = CountVectorizer(analyzer=clean_text) 
countVector = countVectorizer.fit_transform(dataframe['Text_lammetization'])
print('{} Number of Text has {} words'.format(countVector.shape[0], countVector.shape[1]))
#print(countVectorizer.get_feature_names())

1566 Number of Text has 1425 words


In [ ]:
for i in daraframe.iloc[:, 3]:
  print(i)

In [ ]:
# data.to_csv('data_test.csv')

#### Wordcloud Visualization 
ต้องสร้าง target column ก่อน

In [ ]:
#conda install -c conda-forge wordcloud=1.6.0 

In [ ]:
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator